In [42]:
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [5]:
df = pd.read_csv('data/vk_users_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35517 entries, 0 to 35516
Data columns (total 18 columns):
political     5400 non-null float64
country       34114 non-null object
smoking       6264 non-null float64
sex           35517 non-null int64
id            35517 non-null int64
last_name     35517 non-null object
alcohol       6189 non-null float64
religion      6357 non-null object
langs         4624 non-null object
city          32524 non-null object
relation      14826 non-null float64
age           35517 non-null float64
verified      35517 non-null int64
bdate         29173 non-null object
first_name    35517 non-null object
university    14826 non-null float64
life_main     6239 non-null float64
posts         35517 non-null object
dtypes: float64(7), int64(3), object(8)
memory usage: 4.9+ MB


In [26]:
# заменим не указанные значения категориальных признаков на -1
df = df.fillna(-1)
# В столбце university содержится id университета по БД VK. 
# Будем считать, что если он указан, то у пользователя есть высшее 
# образование, иначе нет (хотя строго это не так, он может быть просто 
# не указан. Но примем такое предположение)
df['high_education'] = df['university'].apply(lambda x: 0 if x < -0.5 else 1)
df.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
0,-1.00,Россия,-1.00,1,2615791,Третьякова,-1.00,-1,-1,Москва,-1.00,26.00,0,5.3.1992,Анастасия,-1.00,-1.00,🌸🌸🌸 #id2615791 (fashionlioness) #модель #фотос...,0
1,-1.00,Россия,-1.00,2,148071868,Дмитриев,-1.00,-1,-1,Кострома,-1.00,18.00,0,-1,Сергей,-1.00,-1.00,Карантин)!!!!!!!!!,0
2,3.00,Россия,4.00,1,54774632,Власова,4.00,Православие,-1,Пермь,4.00,110.00,0,6.7,Анюта,0.00,1.00,"Не важно, сколько дверей закроется перед твоим...",1
3,-1.00,Россия,-1.00,1,76303980,Шабалкова,-1.00,-1,-1,Санкт-Петербург,-1.00,90.00,0,3.9,Анастасия,-1.00,-1.00,Друзья! Я собираю большую посылку с помощью д...,0
4,-1.00,Россия,1.00,2,104199626,Блейх,1.00,Православие,"['Русский', 'English']",Санкт-Петербург,1.00,26.00,0,6.11,Эдгар,1.00,1.00,Не мой среди твоих\nИстинный ариец. Характер —...,1


In [27]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [28]:
df_train.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
21023,-1.00,Украина,-1.00,2,167972799,Божко,-1.00,-1,-1,Першотравенск,-1.00,14.00,0,27.2,Богдан,-1.00,-1.00,С наступающим Новым Годом!!!))\nБратишка😎 \nПа...,0
24997,666.00,Беларусь,4.00,2,37145513,Мороз,4.00,-1,-1,Гродно,0.00,26.00,0,6.9.1993,Александр,19637.00,-1.00,"А вы знали?\nВот это качество, вот что значит ...",1
29595,-1.00,Чехия,-1.00,2,89174716,Krasovskiy,-1.00,-1,-1,Praha,0.00,106.00,0,12.6.1913,Alex,141866.00,-1.00,We do not remember days. we remember moments.\...,1
27560,-1.00,Россия,-1.00,1,857985,Сураева,-1.00,-1,-1,Санкт-Петербург,-1.00,38.00,0,25.4.1979,Виктория,-1.00,-1.00,Котики умеют расслабляться всей семьёй)\nСо мн...,0
23876,-1.00,Россия,-1.00,1,27942106,Панченко,-1.00,-1,-1,Нижний Новгород,-1.00,98.00,0,13.7,Валерия,-1.00,-1.00,"❤️❤️❤️\nГоржусь нашей командой и тем, что у на...",0


In [29]:
df_test.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education
12562,6.00,Украина,2.00,2,99952280,Khimich,2.00,-1,-1,Херсон,0.00,106.00,0,16.6,Serhii,13215.00,-1.00,С Новым годом Украинооо))\nТвоя школа...\n)\nО...,1
18369,-1.00,Россия,-1.00,1,36834050,Огаркова,-1.00,-1,-1,Ставрополь,-1.00,30.00,0,8.7.1989,Раисия,-1.00,-1.00,7 миллиардов людей\n\n14 миллиардов лиц 🎭\n💎\n...,0
12482,-1.00,США,-1.00,2,27362004,Оксузян,-1.00,-1,-1,Los Angeles,-1.00,94.00,0,30.11,Эдвард,-1.00,-1.00,...,0
23025,-1.00,-1,-1.00,2,237768,Istomin,-1.00,-1,-1,-1,-1.00,90.00,0,-1,Zorian,-1.00,-1.00,Новый кайфовый. Всем любовь.♥ #2018\n\n \nКриз...,0
29320,-1.00,Россия,-1.00,1,66647843,Рогожина,-1.00,-1,-1,Симферополь,-1.00,86.00,0,28.10.1929,Алина,-1.00,-1.00,т.т\nЖдём)\n#Спасибо_Терри .третье место\nНам ...,0


Смысл значений в столбцах sex, political, smoking, alcohol, relation, life_main следующий (из описания API VK):

1) sex - пол пользователя:
1 - женский;
2 - мужской;
0 - пол не указан

2) political - политические предпочтения:

1 - коммунистические;
2 - социалистические;
3 - умеренные;
4 - либеральные;
5 - консервативные;
6 - монархические;
7 - ультраконсервативные;
8 - индиффирентные;
9 - либертарианские;

3) smoking, alcohol - отношение к курению, алкоголю:

1 - резко негативное;
2 - негативное;
3 - компромиссное;
4 - нейтральное;
5 - положительное;

4) relation - семейное положение:

1 - не женат/не замужем;
2 - есть друг/есть подруга;
3 - помолвлен/помолвлена;
4 - женат/замужем;
5 - всё сложно;
6 - в активном поиске;
7 - влюблён/влюблена;
8 - в гражданском браке;
0 - не указано;

5) life_main - главное в жизни:

1 - семья и дети;
2 - карьера и деньги;
3 - развлечения и отдых;
4 - наука и исследования;
5 - совершенствование мира;
6 - саморазвитие;
7 - красота и искусство;
8 - слава и влияние

Попытаемся предсказать возраст (по данным VK (скорее всего, может быть с ошибками: неизвестно, как он там определяется)) по постам пользователя, его полу (пол указан всегда) и указанным признаками из раздела personal, а также по наличию высшего образования

#### Функции преобразования постов пользователя

In [70]:
from nltk.corpus import stopwords

stopwords_en = stopwords.words('english')
stopwords_ru = stopwords.words('russian')
stopwords_ge = stopwords.words('german')

stopwords_all = stopwords_en + stopwords_ru + stopwords_ge

In [117]:
def tokenizer(text):
    text = text.decode('utf-8')
    text = re.sub('<[^>]*>', '', text).lower()
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub(ur'\W', ' ', text, flags=re.U) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stopwords_all]
    return tokenized

In [32]:
def get_posts_col(df):
    return df[['posts']]

def get_categorial_cols(df):
    return df[['sex', 'high_education', 'political', 'smoking', 'alcohol', 'relation', 'life_main']]

vec = make_union(*[
    make_pipeline(FunctionTransformer(get_categorial_cols, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_posts_col, validate=False))
])

In [33]:
x_train = vec.fit_transform(df_train)
x_train.shape

(24861, 8)

In [41]:
get_posts_col(df_test)

,posts
12562,С Новым годом Украинооо))\nТвоя школа...\n)\nО...
18369,7 миллиардов людей\n\n14 миллиардов лиц 🎭\n💎\n...
12482,...
23025,Новый кайфовый. Всем любовь.♥ #2018\n\n \nКриз...
29320,т.т\nЖдём)\n#Спасибо_Терри .третье место\nНам ...
31716,"Счастья!!!\nС Днем Рождения, сестра!!!\n💋 И сн..."
20173,❌💛\nВикинг 💪🏼\n🌹
19033,https://vk.com/avacs6\nhttps://vk.com/wall-105...
33922,Посмотри своих поклонников здесь : \n✨ ⭐ ⭐ ► ►...
5867,[id204530265|ван лав] * 😍\nНа этой планете сущ...
